## Fit and Validate a Logistic Regression

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score,roc_auc_score, auc, roc_curve
from sklearn.multiclass import OneVsRestClassifier

In [2]:
intro = "/media/eric/nachmanides/programming/projects/post-program_project/"
df = pd.read_pickle(intro+"modeling-eric/data/final_df.pkl")

In [3]:
# OPTIONAL: Drop all NAs in the response variable
# df = df[(df.safe == 0.0) | (df.safe == 1.0)]

### Preprocessing

#### Find a manageable way to represent the sunrise and sunset times

In [4]:
# Let the sunrise_delta and sunset_delta be, for the day of each timebin, the elapsed 
# time between midnight of that day and sunrise (respectively, sunset) on that day. 

midnight = df['sunrise_time'].apply(lambda x: x.round(freq = 'd'))
df['sunrise_delta'] = (df['sunrise_time'] - midnight)  / np.timedelta64(1,'m')
df['sunset_delta'] = (df['sunset_time'] - midnight)  / np.timedelta64(1,'m')

In [5]:
# We will now drop sunrise time and sunset time, since those predictors are hard to 
# compare with the others, and NaTs can't be fed into StandardScaler. We will 
# instead use sunrise_delta and sunset_delta as measures of how early or late 
# sunrise and sunset are on each day. 
df = df.drop(['sunrise_time','sunset_time'],axis=1)

#### Further Preprocessing: 
#### 1. Upsample minority class of 0 and 1 to make them even and then upsample both 0 and 1 classes.
#### 2. Differentiate predictors from response
#### 3. Fill predictor NAs with column means
#### 4. Standardize predictors
#### 5. Binarize categorical predictors

In [6]:
# to deal with unbalanced data, try upsampling
from sklearn.utils import resample

# Separate majority and minority classes
df_mj = df[df.safe==0.0] # safe
df_mi = df[df.safe==1.0] # unsafe
df_na = df[df.safe==2.0] # don't know
 
# Upsample minority class, with replacement 
df_mi_up = resample(df_mi, replace=True, n_samples=len(df_na), random_state=42) 
df_mj_up = resample(df_mj, replace=True, n_samples=len(df_na), random_state=42) 

# Combine majority class with upsampled minority class
df_up= pd.concat([df_mj_up, df_mi_up, df_na])

# Re-sort index so our timebins are ordered again
df_up = df_up.sort_index()

# Set variable 'df' to df_up
df = df_up

# Display new class counts
df.safe.value_counts()

0.0    1878
1.0    1878
2.0    1878
Name: safe, dtype: int64

In [7]:
# Get just the unprocessed predictor columns
pred = df.drop('safe',axis=1)

# Impute means of each column
pred = pred.fillna(pred.mean())

# Standardize the predictors
scaler = StandardScaler()
scaler.fit(pred)

# # Define the predictors that we will train on
X = scaler.transform(pred)

# Binarize the response variable classes and save the result as our response "column(s)"
y = df.safe
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]

### Split, Train, and Validate our Classifier

In [8]:
# Define our training and test sets
# NOTE: Because we have time series, we can't split our train and test sets randomly.
tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)

# Call our classifier. Since we're keeping NAs (in the form of 2.0), 
# we'll treat this as a multi-class classification problem. 
clf = OneVsRestClassifier(LogisticRegression(multi_class = 'ovr', 
                                                random_state = 42))


# Create dicts whose keys are the classes and whose values 
# are the arrays of scores for those classes for our metrics

acc = dict()
prec = dict()
rec = dict()
roc_auc = dict()
fpr = dict()
tpr = dict()

for i in range(n_classes):
        # Initialize the score arrays for class i
        acc[i] = []
        prec[i] = []
        rec[i] = []
        roc_auc[i] = []
        fpr[i] = []
        tpr[i] = []

for train_index, test_index in tscv.split(X):
    
    # Define indices identifying each pair of CV training and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit our classifier: 
    clf_fit = clf.fit(X_train, y_train)
    
    # Learn to predict each class against the other
    y_score = clf_fit.predict(X_test)
    y_pred_proba = clf_fit.predict_proba(X_test)
    
    # For each response class and each metric, append our classifier's 
    # score for that class w.r.t. that metric, on the train-test-split
    # under consideration:
    for i in range(n_classes):
        
        # Calculate the scores for class i
        accuracy = accuracy_score(y_test[:, i], y_score[:, i])
        precision = precision_score(y_test[:, i], y_score[:, i])
        recall = recall_score(y_test[:, i], y_score[:, i])
        false_pos, true_pos, _ = roc_curve(y_test[:, i], y_pred_proba[:, i])
        area_under_curve = auc(false_pos,true_pos)
        
        acc[i].append(accuracy)
        prec[i].append(precision)
        rec[i].append(recall)
        roc_auc[i].append(area_under_curve)
        fpr[i].append(false_pos)
        tpr[i].append(true_pos)

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22.

In [9]:
# Define a function that aggregates a measure across CV folds
def aggregate_across_CVfolds(measure):
    measure_df = pd.DataFrame(measure)
    measure_agg = []
    
    for i in range(0,len(measure_df.columns)):
        measure_agg.append(measure_df.iloc[:,i].mean())
        
    return measure_agg

fpr_agg = []
tpr_agg = []

# Replace each array by its mean (that is, mean CV score)
for i in range(n_classes):
    acc[i] = np.mean(acc[i])
    prec[i] = np.mean(prec[i])
    rec[i] = np.mean(rec[i])
    roc_auc[i] = np.mean(roc_auc[i])

    fpr_agg.append(aggregate_across_CVfolds(fpr[i]))
    tpr_agg.append(aggregate_across_CVfolds(tpr[i]))

        
    print("\nresponse class:\t", i, 
          "\naccuracy:\t",acc[i],
          "\nprecision:\t",prec[i],
          "\nrecall:\t\t",rec[i],
          "\nROC AUC:\t",roc_auc[i])


response class:	 0 
accuracy:	 0.5929712460063898 
precision:	 0.34844949262137476 
recall:		 0.3200594145898247 
ROC AUC:	 0.5266844680158627

response class:	 1 
accuracy:	 0.7284345047923322 
precision:	 0.4767596016548897 
recall:		 0.33197239612976664 
ROC AUC:	 0.5422328693554712

response class:	 2 
accuracy:	 0.5218317358892438 
precision:	 0.30742934138111233 
recall:		 0.6396281481800932 
ROC AUC:	 0.49963781581245853


#### Plot ROC Curves:

In [10]:
# Plot of a ROC curve for each response class on each cross validation fold
import matplotlib.pyplot as plt
plt.style.use("ggplot")

# For each response class i
for i in range(0,3):
        
    # Plot the ROC curve for class i
    plt.plot(fpr_agg[i], tpr_agg[i], color='darkorange',
    lw=2, label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve for Response Class '+str(i))
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(intro + "modeling-eric/visualizations/upsampled_multi_class/ROC_" + "resp_category"+str(i)+".png")
    plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>